## Create npz (if needed)

We'll skip this for now since we already have a lot of npzs created. Also, the fig8 image download pipeline includes create_training_data. This part would mostly be for creating npz files from model outputs.

## Create trk (if needed)
Similar to creating npz. This part could probably take in an npz or the raw images + annotations. Adding an existing lineage should be optional. Currently, deepcell.org returns raw, annotations, and lineage zipped together (so that would be the lineage that goes into the trk), but this part could also just make a lineage analogous to Caliban's save_as_trk option.

## Split npz into pieces (semi-optional)
The idea is that you'd be starting from one huge npz and breaking it into managable pieces. This part will probably include:
- reshape npz (size of each piece is smaller, but same number of frames)
- break up each npz into fewer frames (annotator does not necessarily need to do all 30+ frames of a movie at once)
- save these reshaped pieces as individual npz files, so they can be uploaded and worked on separately
- relabel the npzs as needed (choose between no relabel, relabel each cell in each frame to have unique label, or relabel to have unique labels but preserve relationships across 

These pieces will need specific names so that we can put them back together again if needed (especially putting frames back into longer contiguous movies).

(parts of this are already written - Geneva)

## Upload pieces to AWS
Go through folder of npz (or trk) pieces and upload to a folder in caliban-input bucket. Should be mostly modeled off of aws_upload in pre-processing pipeline. Once each file gets uploaded, the url that caliban will use should get added to a list. That url needs to be constructed so that caliban, during the fig8 job, can access the file.

## Create csv and upload it to fig8
Should be very similar to pre-annotation pipeline. Fig8 needs a csv where each row represents one task. In this case, the important information is the link to caliban (with the file name) under the column "project_url". After that csv gets created, an existing fig8 job gets copied (no data gets copied over, just instructions & settings), and the csv gets uploaded to the new job.

In [ ]:
job_id_to_copy =  1432466 #Caliban test job